In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('/home/lichi/jupiter/imaginarium')
from random import sample

from PIL import Image
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
import keras.backend as K
from keras.layers import Conv1D,Activation, Reshape, Conv2D,Input, Dense, dot, multiply, add, concatenate, Lambda
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import mean_squared_error

from visual import tools, evaluation

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Пишем сетку

Берем 5 случайных картинок, из них выбираем одну

1) y - ohe номера картинки. 

2) X1 -  расстояние от картинки до топ 5 слов( часть речи, длина этих слов, сколько раз эти слова встречаются в топ-5 других картинок)

3) X2 - расстояние от полученного среднего вектора слов до 5 картинок, сколько раз картинки встречаются в топ-5 этих слов

4) Лосс - логлосс при обучении 2 игрока, при обучении 1 игрока - 1.5*p*(1-p)*(1-p) + p*p*(1-p)*1.5




### Читаем данные, обработанные до этого

In [11]:
import pickle

with open('preprocessed_data/words_coco_vectors.pickle', 'rb') as f:
    words_vectors = pickle.load( f)
with open('preprocessed_data/image_coco_vectors.pickle', 'rb') as f:
    image_vectors = pickle.load( f)
    
    
# Дампим массив, в котором содержатся индексы  топ10,
# близких по mse слов-ассоциаций к картинке для каждой картинки
with open('preprocessed_data/idx_words_array.pickle', 'rb') as f:
    idx_words_array = pickle.load( f)
# Дампим массив, в котором содержатся индексы  топ10,
# близких по mse слов-ассоциаций к картинке для каждого ckjdf
with open('preprocessed_data/idx_images_array.pickle', 'rb') as f:
    idx_images_array = pickle.load( f)
# Дампим массив, в котором содержатся инфа о том,
# в скольки процентах случаев это слово входит в топ-10 ближайших для всех картинок
with open('preprocessed_data/norm_idx_words_array.pickle', 'rb') as f:
    norm_idx_words_array = pickle.load( f)
# Дампим массив, в котором содержатся инфа о том,
# в скольки процентах случаев эта картинка входит в топ-10 ближайших для всех слов
with open('preprocessed_data/norm_idx_images_array.pickle', 'rb') as f:
    norm_idx_images_array = pickle.load( f)
# Дампим массив, в котором содержатся расстояние до топ k слов 
with open('preprocessed_data/mse_words_top_array.pickle', 'rb') as f:
    mse_words_top_array = pickle.load( f)
# Дампим массив, в котором содержатся расстояние до топ k слов 
with open('preprocessed_data/mse_words_top_array.pickle', 'rb') as f:
    mse_words_top_array = pickle.load( f)

### Обучающая выборка

In [9]:
X_ind, y_ind, y, X1 = [], [], [], []
X_img_freq = []
num_samples = 100000
num_images = idx_words_array.shape[0]
num_words = idx_words_array.shape[1]
k = 10
for i in range(num_samples):
    X_ind.append(np.array(sample(range(1,num_images), k)))
    y_ind.append(sample(X_ind[i], 1))
    y.append(np.array([1 if x==y_ind[i] else 0 for x in X_ind[i]]))
    dist_words = mse_words_top_array[y_ind[i]]
    freq_word = norm_idx_words_array[np.where(idx_words_array[0]==1)]
    X1.append(np.concatenate([dist_words.reshape(k,1), freq_word.reshape(k,1)], axis = 1))
    freq_image = norm_idx_images_array[np.where(idx_images_array[0]==1)]
    X_img_freq.append(freq_word)

In [12]:
X2 = image_vectors[X_ind].reshape(num_samples,1024,k)
X1 = np.array(X1).reshape(num_samples,2,k)
y = np.array(y).reshape(num_samples,1,k)
X3 = X2.reshape(num_samples,k,1024)
X30 = X3[:,0,:].reshape(num_samples,1024)
X31 = X3[:,1,:].reshape(num_samples,1024)
X_img_freq = np.array(X_img_freq)

### Сетка, которая пока просто подбирает идеальное слово под случайно выбранную картинку для 1 агента,  а потом 2 агент просто выбирает ближайшее слово (или учит как выбирать dense слоем (тоже такой штраф за небанальность))

In [13]:
def mean_squared_error_keras(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=2)
k = 10


In [16]:

# Сетка, которая пока просто подбирает идеальное слово под случайно выбранную картинку для 1 агента, 
# а потом 2 агент просто выбирает ближайшее слово

# Входы 10 ближайших пар, заранее сгенерированные расстояния между ближайшими словами и картинками, и частоту топ 10 
# близости каждого слова к ближайшим 10 картинкам
inputs1 = Input(shape=(2,k))
# Эмбеддинги для слов
inputs2 = Input(shape=(1024,k))
inputs21 = Input(shape=(k,1024))
# OHE-вектор, какое слово было истинным
inputs3 = Input(shape=(1,k))
#  Учим свертку, которая делает штраф-поощрения за банальность (дистанция между словами, попсовость слов)
# На выходе получили веса для каждого слова
conv1d = Conv1D(k, (2), activation='softmax', padding = 'valid')(inputs1)
#predictions = Dense(5, activation='softmax')(conv1d)
# усредняем слова с весами, выученными сверткой
merge_layer = multiply([conv1d, inputs2])
# делаем повтор, чтобы потом считать новые минимальные дистанции
sum_layer = Lambda(lambda x: K.repeat(K.sum(x, axis = 2), k), output_shape=(1024, k))(merge_layer)

# считаем разницу между полученными средними одного и другого
distance_layer = Lambda(lambda x: mean_squared_error_keras(x[0], x[1]),
                        output_shape=(1,k))([inputs21, sum_layer])

#features_img = concatenate([distance_layer, inputs3], axis = 1)
#output = Conv1D(5, (2), activation='softmax', padding = 'valid')(features_img)

dense_overfit = Dense(100, activation='relu')(distance_layer)
# Выбираем правильный ответ (аналог агента)
output = Dense(k, activation='softmax')(dense_overfit)

#подбираем картинки
model_keras = Model(inputs=[inputs1, inputs2, inputs21], outputs=output)
model_keras.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])





In [ ]:
model_keras.fit([X1, X2, X2.reshape(num_samples, k,1024)],y, epochs = 10, 
                verbose = True, batch_size = 500, )

Epoch 1/10
100000/100000 [==============================] - 59s 586us/step - loss: 115.1315 - acc: 0.1002
Epoch 2/10
100000/100000 [==============================] - 54s 544us/step - loss: 115.1304 - acc: 0.1002
Epoch 3/10
 42500/100000 [===========>..................] - ETA: 30s - loss: 115.1317 - acc: 0.0994

#### Отрисовать сетку

In [ ]:
from keras.utils import plot_model
plot_model(model_hard, to_file='model_hard.png')

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model_keras).create(prog='dot', format='svg' ))
